In [14]:
import pytest
from unittest.mock import patch
import re
import logging.config
# from transformers import AutoModelForTokenClassification, pipeline, AutoTokenizer
import transformers
from transformers.pipelines import PIPELINE_REGISTRY
import threading
import time
from pipeline import NER_Pipeline
from queue import Queue
import streamlit as st
from annotated_text import annotated_text
from collections import Counter, OrderedDict
import pandas as pd
import altair as alt
import wikipediaapi
import re
import wikipedia
import requests
import json
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.keys import Keys

from server import get_acronym_description, request_results, get_acronyms_and_long_forms

In [2]:

# Register custom pipeline
PIPELINE_REGISTRY.register_pipeline(
    "NER_NLP_tagger",
    pipeline_class=NER_Pipeline,
    pt_model=transformers.AutoModelForTokenClassification
)

# Load NER tagger pipeline
ner_tagger = transformers.pipeline(
    "NER_NLP_tagger", model="SurtMcGert/NLP-group-CW-roberta-ner-tagging")

In [3]:
def compare(expected, predicted):
    if (expected == predicted):
        print("============ Test passed ============")
    else:
        print(f"============ Test failed ============\nPredicted:\n{predicted}")

In [4]:
# Test to see if ner_tagger is correctly called
input = "This is a test"

expected = ner_tagger(input)

predicted = request_results(input)

compare(expected, predicted)

[nltk_data] Downloading package wordnet to C:\Users\Chris
[nltk_data]     Harland\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


============ Test passed ============


[nltk_data] Downloading package wordnet to C:\Users\Chris
[nltk_data]     Harland\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [5]:
# Test to see if get_acronym_description is functional
input = "Federal Bureau of Investigation (FBI)"

expected = "The Federal Bureau of Investigation (FBI) is the domestic intelligence and security service of the United States and its principal federal law enforcement agency"

predicted = get_acronym_description(input)

if (predicted == ""):
    print(True)
else:
    compare(expected, predicted)

============ Test passed ============


In [6]:
# Test to see if get_acronyms_and_long_forms is functional
input = [("This", "B-O"), ("is", "B-O"), ("a", "B-O"), ("test", "B-O"), ("Federal", "B-LF"), ("Bureau", "I-LF"), ("of", "I-LF"), ("Investigation", "I-LF"), ("FBI", "B-AC")]

expected = {
    "Acronyms": ["FBI"],
    "Long Forms": ["Federal Bureau of Investigation"]
}

predicted = get_acronyms_and_long_forms(input)

compare(expected, predicted)

============ Test passed ============


In [22]:
# Test server's responsiveness

browser = webdriver.Firefox()

browser.get("http://localhost:8501")

time.sleep(5)

element = WebDriverWait(browser, 20).until(EC.element_to_be_clickable((By.XPATH, "//*[@id=\"text_input_1\"]")))
element.send_keys("This is a test")
element.send_keys(Keys.RETURN)

xpaths = [
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[6]/div[2]/div/div/div/div/div/div/table",
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[4]/div/div/p/span",
    "/html/body/div[1]/div[1]/div[1]/div/div/div/section/div[1]/div/div/div/div[6]/div[2]/div/div/div/div/div/div/table"
]

test_pass = True
for xpath in xpaths:
    try:
        element = WebDriverWait(browser, 20).until(EC.presence_of_element_located((By.XPATH, xpath)))
    except:
        test_pass = False

browser.quit()

if (test_pass == True):
    print("Test passed")
else:
    print("Test failed")
